RUHAAN HAWALDAR BE 21137

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
%%writefile max_cuda.cu

#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>

__global__ void max1(int* input, int size) {
    int tid = threadIdx.x;

    for (int step_size = 1; step_size < size; step_size *= 2) {
        if (tid % (2 * step_size) == 0) {
            int first = tid;
            int second = tid + step_size;

            if (second < size) {
                if (input[first] < input[second]) {
                    input[first] = input[second];
                }
            }
        }
        __syncthreads();
    }
}

int main() {
    const int count = 8;
    const int size = count * sizeof(int);
    int h[count] = {13, 65, 15, 14, 33, 2, 30, 8};

    int* d;
    cudaMalloc(&d, size);
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

    max1<<<1, count / 2>>>(d, count);

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

    printf("Largest number is: %d\n", result);

    cudaFree(d);
    return 0;
}

Writing max_cuda.cu


In [4]:
!nvcc max_cuda.cu -o max_cuda

In [5]:
!./max_cuda

Largest number is: 13


In [6]:
%%writefile min_cuda.cu

#include <stdio.h>
#include <cuda.h>
#include <stdlib.h>

__global__ void min1(int* input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    int temp;

    while (number_of_threads > 0) {
        if (tid < number_of_threads) {
            int fst = tid * step_size * 2;
            int snd = fst + step_size;

            if (input[fst] > input[snd]) {
                temp = input[fst];
                input[fst] = input[snd];
                input[snd] = temp;
            }
        }
        __syncthreads();
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

int main() {
    const int count = 8;
    const int size = count * sizeof(int);
    int h[] = {13, 65, 15, 14, 33, 23, 30, 8};

    int* d;
    cudaMalloc(&d, size);
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

    min1<<<1, count / 2>>>(d);

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

    printf("Smallest number is: %d\n", result);

    cudaFree(d);
    return 0;
}


Writing min_cuda.cu


In [7]:
!nvcc min_cuda.cu -o min_cuda
!./min_cuda

Smallest number is: 13


In [8]:
%%writefile sum_avg_cuda.cu

#include <stdio.h>
#include <cuda.h>

__global__ void sum(int* input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;

    while (number_of_threads > 0) {
        if (tid < number_of_threads) {
            int fst = tid * step_size * 2;
            int snd = fst + step_size;
            input[fst] += input[snd];
        }
        __syncthreads();
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

int main() {
    const int count = 8; // Total elements
    const int size = count * sizeof(int);
    int h[] = {13, 27, 15, 14, 33, 2, 30, 8}; // Input array

    int* d;
    cudaMalloc(&d, size);
    cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

    sum<<<1, count / 2>>>(d); // Call kernel function

    int result;
    cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

    float avg = (float)result / count; // Calculate average

    printf("Sum is %d\n", result);
    printf("Average is %.2f\n", avg);

    cudaFree(d);
    return 0;
}

Writing sum_avg_cuda.cu


In [9]:
!nvcc sum_avg_cuda.cu -o sum_avg_cuda
!./sum_avg_cuda

Sum is 13
Average is 1.62
